## follow

In [10]:
import pandas as pd

### get data

In [ ]:
import pandas as pd

In [ ]:
# read finalTable.csv all columns as string
finalTable = pd.read_csv('finalTable.csv', dtype=str)
finalTable

In [ ]:
# give me unique values of twitter_id and author_id
print(len(finalTable['twitter_id'].unique()))
print(len(finalTable['author_id'].unique()))

In [ ]:
import sqlite3

# connect to SQLite database
conn = sqlite3.connect('twitterFollows.db')  
c = conn.cursor()

# create tables in the SQLite database
c.execute('''
    CREATE TABLE IF NOT EXISTS FollowData
    (User1 TEXT NOT NULL,
    User2 TEXT NOT NULL,
    FollowEachOther BOOLEAN NOT NULL);
''')

c.execute('''
    CREATE TABLE IF NOT EXISTS UserFollowing
    (UserId TEXT NOT NULL,
    FollowingId TEXT NOT NULL);
''')



In [ ]:
import tweepy
import pandas as pd

def get_following(id, client):
    following_ids = []
    for page in tweepy.Paginator(client.get_users_following, id, max_results=1000):
        following_ids.extend([user['id'] for user in page.data])
    return following_ids

user_ids = finalTable['author_id'][0:2].tolist()  # Replace with your list of user ids

print(f"Found {len(user_ids)} users to process.")

# Replace with your Bearer token
client = tweepy.Client(bearer_token="AAAAAAAAAAAAAAAAAAAAAGYKkQEAAAAAbNiwJ4AF4PTE8km%2FFTVoB7ewbds%3Dv7YaQmbfxG4r6NnkRvvqfeysnZ0ylCf3EIOHHiKzMRxomN8w4l", wait_on_rate_limit=True)

user_following = {user_id: get_following(user_id, client) for user_id in user_ids}
        
for user_id in user_ids:
    print(f"Fetching following ids for user {user_id}.")
    following = get_following(user_id, client)
    user_following[user_id] = following
    print(f"User {user_id} is following {len(following)} users.")
    # Insert the following ids into the UserFollowing table
    for following_id in following:
        c.execute("INSERT INTO UserFollowing (UserId, FollowingId) VALUES (?, ?)", 
                  (user_id, following_id))
        

print("Finished fetching following ids for all users.")
        
for i in range(len(user_ids)):
    for j in range(i+1, len(user_ids)):
        print(f"Checking if users {user_ids[i]} and {user_ids[j]} follow each other.")
        result = user_ids[j] in user_following[user_ids[i]] and user_ids[i] in user_following[user_ids[j]]
        if result:
            print(f"Users {user_ids[i]} and {user_ids[j]} follow each other.")
        else:
            print(f"Users {user_ids[i]} and {user_ids[j]} do not follow each other.")
        c.execute("INSERT INTO FollowData (User1, User2, FollowEachOther) VALUES (?, ?, ?)", 
                  (user_ids[i], user_ids[j], result))

        
print("Finished processing all users.")

# commit your changes and close the connection
conn.commit()
conn.close()

In [ ]:
import tweepy
import pandas as pd

def get_following(id, client):
    following_ids = []
    for page in tweepy.Paginator(client.get_users_following, id, max_results=1000):
        following_ids.extend([user['id'] for user in page.data])
    return following_ids

# Replace with your Bearer token
client = tweepy.Client(bearer_token="AAAAAAAAAAAAAAAAAAAAAGYKkQEAAAAAbNiwJ4AF4PTE8km%2FFTVoB7ewbds%3Dv7YaQmbfxG4r6NnkRvvqfeysnZ0ylCf3EIOHHiKzMRxomN8w4l", wait_on_rate_limit=True)

user_ids = finalTable['author_id'].tolist()  # Replace with your list of user ids

user_following = {user_id: get_following(user_id, client) for user_id in user_ids}

results = []
for i in range(len(user_ids)):
    for j in range(i+1, len(user_ids)):
        result = user_ids[j] in user_following[user_ids[i]] and user_ids[i] in user_following[user_ids[j]]
        results.append([user_ids[i], user_ids[j], result])

df = pd.DataFrame(results, columns=['User1', 'User2', 'FollowEachOther'])

print(df)

In [ ]:
#final code run on digitalocean

import tweepy
import pandas as pd
import sqlite3
import logging

logging.basicConfig(filename='log.txt', level=logging.INFO)

def get_following(id, client):
    following_ids = []
    for page in tweepy.Paginator(client.get_users_following, id, max_results=1000):
        following_ids.extend([user['id'] for user in page.data])
    return following_ids

# connect to SQLite database
conn = sqlite3.connect('twitter_data.db')
c = conn.cursor()

# create tables in the SQLite database
c.execute('''
    CREATE TABLE IF NOT EXISTS FollowData
    (User1 TEXT NOT NULL,
    User2 TEXT NOT NULL,
    FollowEachOther BOOLEAN NOT NULL);
''')

c.execute('''
    CREATE TABLE IF NOT EXISTS UserFollowing
    (UserId TEXT NOT NULL,
    FollowingId TEXT NOT NULL);
''')

# Load data from CSV
finalTable = pd.read_csv('finalTable.csv', dtype=str)

# Replace with your list of user ids
author_id = finalTable['author_id'].unique().tolist()
twitter_id = finalTable['twitter_id'].unique().tolist()

#combine two list
user_ids = author_id + twitter_id

# get uniwue user_ids
user_ids = list(set(user_ids))

logging.info(f"Found {len(user_ids)} users to process.")

# Replace with your Bearer token
client = tweepy.Client(bearer_token="AAAAAAAAAAAAAAAAAAAAAGYKkQEAAAAAbNiwJ4AF4PTE8km%2FFTVoB7ewbds%3Dv7YaQmbfxG4r6NnkRvvqfeysnZ0ylCf3EIOHHiKzMRxomN8w4l",
                       wait_on_rate_limit=True)

user_following = {user_id: get_following(user_id, client) for user_id in user_ids}

for user_id in user_ids:
    logging.info(f"Fetching following ids for user {user_id}.")
    following = get_following(user_id, client)
    user_following[user_id] = following
    logging.info(f"User {user_id} is following {len(following)} users.")
    # Insert the following ids into the UserFollowing table
    for following_id in following:
        c.execute("INSERT INTO UserFollowing (UserId, FollowingId) VALUES (?, ?)",
                  (user_id, following_id))

logging.info("Finished fetching following ids for all users.")

for i in range(len(user_ids)):
    for j in range(i + 1, len(user_ids)):
        logging.info(f"Checking if users {user_ids[i]} and {user_ids[j]} follow each other.")
        result = user_ids[j] in user_following[user_ids[i]] and user_ids[i] in user_following[user_ids[j]]
        if result:
            logging.info(f"Users {user_ids[i]} and {user_ids[j]} follow each other.")
        else:
            logging.info(f"Users {user_ids[i]} and {user_ids[j]} do not follow each other.")
        c.execute("INSERT INTO FollowData (User1, User2, FollowEachOther) VALUES (?, ?, ?)",
                  (user_ids[i], user_ids[j], result))

logging.info("Finished processing all users.")

# commit your changes and close the connection
conn.commit()
conn.close()

### create matrix

In [47]:
# Load data from CSV
finalTable = pd.read_csv('finalTable.csv', dtype=str)

# Replace with your list of user ids
author_id = finalTable['author_id'].unique().tolist()
twitter_id = finalTable['twitter_id'].unique().tolist()

#combine two list
user_ids = author_id + twitter_id

# get uniwue user_ids
user_ids = list(set(user_ids))

In [48]:
len(user_ids)

118

In [13]:
#close sqlite connection
conn.close()

# read UserFollowing table from followersData.db into df
conn = sqlite3.connect('followersData.db')
following = pd.read_sql_query("SELECT * FROM UserFollowing", conn)
following

,UserId,FollowingId
0,1048784496,186613457
1,1048784496,31200707
2,1048784496,242555999
3,1048784496,310421696
4,1048784496,1099199839
...,...,...
218838,720353279176081408,44174176
218839,720353279176081408,47171023
218840,720353279176081408,56179208
218841,720353279176081408,14335435


In [33]:
conn.close()
conn = sqlite3.connect("D://Dropbox//projeler//ndsu_polarization//twitterVeriToplama//twitter004.db")
ids = pd.read_sql_query("SELECT author_id, twitter_id FROM authorIDTwitterID", conn)
ids

,author_id,twitter_id
0,1017500185356853248,1017500185356853248
1,102477372,278124059
2,1048784496,1048784496
3,1058520120,1058520120
4,1060370282,1060370282
...,...,...
119,941000686275387392,941000686275387392
120,946549322,946549322
121,968650362,968650362
122,970207298,970207298


In [51]:
import pandas as pd

df = following.copy()

# Merge the dataframe with itself to check for mutual relationships
merged_df = pd.merge(df, df, left_on=['UserId', 'FollowingId'], right_on=['FollowingId', 'UserId'], how='left')

# If the merge is successful, then there's a mutual relationship
merged_df['FollowEachOther'] = ~merged_df['UserId_y'].isnull()

result2 = merged_df[['UserId_x', 'FollowingId_x', 'FollowEachOther']].rename(columns={'UserId_x': 'User1', 'FollowingId_x': 'User2'})

# Drop duplicates
result2.drop_duplicates(inplace=True)

result2


,User1,User2,FollowEachOther
0,1048784496,186613457,False
1,1048784496,31200707,False
2,1048784496,242555999,False
3,1048784496,310421696,False
4,1048784496,1099199839,False
...,...,...,...
218838,720353279176081408,44174176,False
218839,720353279176081408,47171023,False
218840,720353279176081408,56179208,False
218841,720353279176081408,14335435,False


In [52]:
# get all rows where FollowEachOther = 1
result2[result2['FollowEachOther'] == 1]

,User1,User2,FollowEachOther
6,1048784496,515822213,True
7,1048784496,1356714265894408196,True
21,1048784496,19318314,True
22,1048784496,1060370282,True
79,1048784496,7429102,True
...,...,...,...
218694,1074480192,229966028,True
218695,1074480192,216881337,True
218698,1074480192,13218102,True
218699,1074480192,88784440,True


In [53]:
# drop duplicates in result2
result2 = result2.drop_duplicates()
result2

,User1,User2,FollowEachOther
0,1048784496,186613457,False
1,1048784496,31200707,False
2,1048784496,242555999,False
3,1048784496,310421696,False
4,1048784496,1099199839,False
...,...,...,...
218838,720353279176081408,44174176,False
218839,720353279176081408,47171023,False
218840,720353279176081408,56179208,False
218841,720353279176081408,14335435,False


In [54]:
unique_user1 = result2['User1'].unique()
result2 = result2[result2['User2'].isin(unique_user1)]
result2

,User1,User2,FollowEachOther
2,1048784496,242555999,False
4,1048784496,1099199839,False
5,1048784496,1351236267707654145,False
6,1048784496,515822213,True
7,1048784496,1356714265894408196,True
...,...,...,...
218698,1074480192,13218102,True
218699,1074480192,88784440,True
218713,720353279176081408,1074480192,False
218741,720353279176081408,817138492614524928,True


In [55]:
import pandas as pd

df1 = result2.copy()

df2 = ids.copy()

# Merging
merged_df_1 = pd.merge(df1, df2, left_on='User1', right_on='author_id', how='left')
merged_df_1 = merged_df_1.drop('author_id', axis=1).rename(columns={'twitter_id': 'TwitterId_1'})

merged_df_final = pd.merge(merged_df_1, df2, left_on='User2', right_on='author_id', how='left')
merged_df_final = merged_df_final.drop('author_id', axis=1).rename(columns={'twitter_id': 'TwitterId_2'})

merged_df_final


,User1,User2,FollowEachOther,TwitterId_1,TwitterId_2
0,1048784496,242555999,False,1048784496,242555999
1,1048784496,1099199839,False,1048784496,1099199839
2,1048784496,1351236267707654145,False,1048784496,1351236267707654145
3,1048784496,515822213,True,1048784496,515822213
4,1048784496,1356714265894408196,True,1048784496,1356714265894408196
...,...,...,...,...,...
5364,1074480192,13218102,True,1074480192,13218102
5365,1074480192,88784440,True,1074480192,88784440
5366,720353279176081408,1074480192,False,817138492614524928,1074480192
5367,720353279176081408,817138492614524928,True,817138492614524928,817138492614524928


In [57]:
import pandas as pd
import numpy as np

df = merged_df_final.copy()

# Filter 'user2' values based on 'user1' unique values
# unique_user1 = df['User1'].unique()
# df = df[df['User2'].isin(unique_user1)]

# Get unique users from the filtered dataframe
users = list(set(df['TwitterId_1'].tolist() + df['TwitterId_2'].tolist()))

# Create an empty matrix of zeros
matrix = np.zeros((len(users), len(users)))

# Populate the matrix
for index, row in df.iterrows():
    i = users.index(row['TwitterId_1'])
    j = users.index(row['TwitterId_2'])
    if row['FollowEachOther']:
        matrix[i][j] = 1
        matrix[j][i] = 1  # If they follow each other, it's a symmetric relationship

# Convert numpy matrix to DataFrame
df_matrix = pd.DataFrame(matrix, index=users, columns=users)

df_matrix

,249787913,1352287997853622273,278124059,1074480192,21406834,15324851,486694111,7429102,811313565760163844,117501995,...,941000686275387392,109287731,13218102,202206694,92186819,131546062,88784440,382791093,1351236267707654145,247334603
249787913,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,...,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0
1352287997853622273,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
278124059,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
1074480192,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
21406834,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131546062,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,...,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
88784440,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
382791093,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,...,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0
1351236267707654145,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,...,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [58]:
# check if a df is symmetric
def is_symmetric(df):
    return df.equals(df.T)

is_symmetric(df_matrix)

True

In [61]:
#save df_dfmatrix to sqlite database named csvyerine.db close previous connections
conn.close()
conn = sqlite3.connect('csvyerine.db')
df_matrix.to_sql('followingMatrixAll', con=conn, if_exists='replace', index=True)
conn.close()

## mention & reply ???

In [1]:
import sqlite3
conn = sqlite3.connect("D://Dropbox//projeler//ndsu_polarization//twitterVeriToplama//twitter004.db")
c = conn.cursor()
c

### create mentioned accounts table (once)

In [ ]:
# create new table
c.execute('''
    CREATE TABLE mentionedUsers3 (
        author_id TEXT,
        twitter_id TEXT,
        mentioned_id TEXT,
        state TEXT,
        party TEXT
    )
''')

In [2]:
# query data from finalTweetsTable
c.execute('SELECT author_id, twitter_id, state, party, entities FROM finalTweetsTable')
rows = c.fetchall()

In [3]:
rows[33]

('45645232',
 '224285242',
 'CO',
 'Democrat',
 '{"urls": [{"start": 66, "end": 89, "url": "https://t.co/uLu9rpj9MV", "expanded_url": "https://secure.actblue.com/donate/mfb-soc-2022?refcode=tw-221031", "display_url": "secure.actblue.com/donate/mfb-soc…", "images": [{"url": "https://pbs.twimg.com/news_img/1587194490879504384/JtRfySe_?format=jpg&name=orig", "width": 1200, "height": 657}, {"url": "https://pbs.twimg.com/news_img/1587194490879504384/JtRfySe_?format=jpg&name=150x150", "width": 150, "height": 150}], "status": 200, "title": "Chip in to support Michael Bennet", "description": "Make a donation now to help Michael fight off the GOP and protect the Democratic majority.", "unwound_url": "https://secure.actblue.com/donate/mfb-soc-2022?refcode=tw-221031"}]}')

In [ ]:
# parse JSON and insert data into new table
import json
for row in rows:
    author_id, twitter_id, state, party, entities = row
    if entities is not None:
        try:
            entities_json = json.loads(entities)
            for mention in entities_json.get('mentions', []):
                mentioned_id = mention.get('id')
                c.execute('INSERT INTO MentionedUsers3 (author_id, twitter_id, state, party, mentioned_id) VALUES (?, ?, ?, ?, ?)', (author_id, twitter_id, state, party, mentioned_id))
        except json.JSONDecodeError:
            print(f"Error parsing JSON for author_id {author_id}: {entities}")

# commit the transactions
conn.commit()

### create matrix

In [10]:
# read * from mentionedUsers table to df
import pandas as pd
mentionedUsers = pd.read_sql_query("SELECT * FROM mentionedUsers3", conn)
#drop state column
mentionedUsers = mentionedUsers.drop(columns=['state'])
mentionedUsers
# burada id sayısı çok fazla çünkü tüm mentionlananlar var. biz sadece kendi listemizdekileri alacağız.

,author_id,twitter_id,mentioned_id,party,region
0,45645232,224285242,4750779374,Democrat,north
1,45645232,224285242,483010877,Democrat,north
2,45645232,224285242,117839957,Democrat,north
3,45645232,224285242,540352897,Democrat,north
4,45645232,224285242,228591530,Democrat,north
...,...,...,...,...,...
10392,1352287997853622273,1352287997853622273,123935925,Democrat,south
10393,1352287997853622273,1352287997853622273,14275261,Democrat,south
10394,1352287997853622273,1352287997853622273,957024441338417154,Democrat,south
10395,1352287997853622273,1352287997853622273,44783853,Democrat,south


In [11]:
authorIDTwitterID = pd.read_sql_query("SELECT * FROM authorIDTwitterID", conn)
authorIDTwitterID = authorIDTwitterID[['author_id', 'twitter_id', 'region', 'party']]
authorIDTwitterID

,author_id,twitter_id,region,party
0,1017500185356853248,1017500185356853248,south,Republican
1,102477372,278124059,north,Democrat
2,1048784496,1048784496,north,Republican
3,1058520120,1058520120,north,Democrat
4,1060370282,1060370282,south,Republican
...,...,...,...,...
119,941000686275387392,941000686275387392,north,Democrat
120,946549322,946549322,north,Democrat
121,968650362,968650362,south,Republican
122,970207298,970207298,north,Democrat


In [ ]:
conn.close()

In [12]:
# mentionedUsers.merge(authorIDTwitterID, left_on = 'mentioned_id', right_on = 'author_id', how='left')
mentionedUsers = mentionedUsers.merge(authorIDTwitterID, left_on='mentioned_id', right_on='author_id', how='left', suffixes=('', '_mentioned'))

In [13]:
mentionedUsers

,author_id,twitter_id,mentioned_id,party,region,author_id_mentioned,twitter_id_mentioned,region_mentioned,party_mentioned
0,45645232,224285242,4750779374,Democrat,north,NaN,NaN,NaN,NaN
1,45645232,224285242,483010877,Democrat,north,NaN,NaN,NaN,NaN
2,45645232,224285242,117839957,Democrat,north,NaN,NaN,NaN,NaN
3,45645232,224285242,540352897,Democrat,north,NaN,NaN,NaN,NaN
4,45645232,224285242,228591530,Democrat,north,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
10392,1352287997853622273,1352287997853622273,123935925,Democrat,south,NaN,NaN,NaN,NaN
10393,1352287997853622273,1352287997853622273,14275261,Democrat,south,NaN,NaN,NaN,NaN
10394,1352287997853622273,1352287997853622273,957024441338417154,Democrat,south,NaN,NaN,NaN,NaN
10395,1352287997853622273,1352287997853622273,44783853,Democrat,south,NaN,NaN,NaN,NaN


In [14]:
print(len(mentionedUsers["mentioned_id"].unique()))
print(len(mentionedUsers["author_id"].unique()))
print(len(mentionedUsers["twitter_id"].unique()))


3479
121
90


In [15]:
# what are the number of unique elements in author_id different than twitter_id?
print(len(mentionedUsers["author_id"].unique()) - len(mentionedUsers["twitter_id"].unique()))

31


In [16]:
# get unique values from 'author_id' column
unique_user1 = mentionedUsers['author_id'].unique()

# filter 'mentioned_id' column based on these unique values
filtered_data = mentionedUsers[mentionedUsers['mentioned_id'].isin(unique_user1)]

In [17]:
print(len(filtered_data))
# buradaki sayı finalTable'daki sayıdan az ama normal. herkes birilerini mentionlamıyor.
print(len(filtered_data['author_id'].unique()))
print(len(filtered_data['twitter_id'].unique()))
print(len(filtered_data['mentioned_id'].unique()))


1805
108
88
113


In [18]:
filtered_data

,author_id,twitter_id,mentioned_id,party,region,author_id_mentioned,twitter_id_mentioned,region_mentioned,party_mentioned
13,45645232,224285242,45645232,Democrat,north,45645232,224285242,north,Democrat
38,45645232,224285242,82489457,Democrat,north,82489457,1351236267707654145,north,Democrat
41,102477372,278124059,102477372,Democrat,north,102477372,278124059,north,Democrat
43,102477372,278124059,102477372,Democrat,north,102477372,278124059,north,Democrat
45,102477372,278124059,102477372,Democrat,north,102477372,278124059,north,Democrat
...,...,...,...,...,...,...,...,...,...
10314,1356714265894408196,1356714265894408196,1356714265894408196,Democrat,south,1356714265894408196,1356714265894408196,south,Democrat
10320,1356714265894408196,1356714265894408196,1356714265894408196,Democrat,south,1356714265894408196,1356714265894408196,south,Democrat
10354,1356714265894408196,1356714265894408196,1356714265894408196,Democrat,south,1356714265894408196,1356714265894408196,south,Democrat
10360,1356714265894408196,1356714265894408196,1048784496,Democrat,south,1048784496,1048784496,north,Republican


In [19]:
(filtered_data["mentioned_id"] == filtered_data["author_id_mentioned"]).sum()

1805

In [20]:
# drop author_id_mentioned column from filtered_data
filtered_data = filtered_data.drop(columns=['author_id_mentioned'])

In [21]:
# rearrange columns
filtered_data = filtered_data[['author_id', 'twitter_id', 'region', 'party', 'mentioned_id', 'twitter_id_mentioned', 'region_mentioned', 'party_mentioned']]
filtered_data

,author_id,twitter_id,region,party,mentioned_id,twitter_id_mentioned,region_mentioned,party_mentioned
13,45645232,224285242,north,Democrat,45645232,224285242,north,Democrat
38,45645232,224285242,north,Democrat,82489457,1351236267707654145,north,Democrat
41,102477372,278124059,north,Democrat,102477372,278124059,north,Democrat
43,102477372,278124059,north,Democrat,102477372,278124059,north,Democrat
45,102477372,278124059,north,Democrat,102477372,278124059,north,Democrat
...,...,...,...,...,...,...,...,...
10314,1356714265894408196,1356714265894408196,south,Democrat,1356714265894408196,1356714265894408196,south,Democrat
10320,1356714265894408196,1356714265894408196,south,Democrat,1356714265894408196,1356714265894408196,south,Democrat
10354,1356714265894408196,1356714265894408196,south,Democrat,1356714265894408196,1356714265894408196,south,Democrat
10360,1356714265894408196,1356714265894408196,south,Democrat,1048784496,1048784496,north,Republican


In [22]:
# kendini mentionlayanları çıkar
filtered_data = filtered_data[filtered_data['author_id'] != filtered_data['mentioned_id']]
filtered_data

,author_id,twitter_id,region,party,mentioned_id,twitter_id_mentioned,region_mentioned,party_mentioned
38,45645232,224285242,north,Democrat,82489457,1351236267707654145,north,Democrat
67,102477372,278124059,north,Democrat,131546062,131546062,south,Republican
72,102477372,278124059,north,Democrat,131546062,131546062,south,Republican
112,3006258669,5558312,south,Republican,5558312,5558312,south,Republican
113,3006258669,5558312,south,Republican,382791093,382791093,north,Republican
...,...,...,...,...,...,...,...,...
10212,1351236267707654145,1351236267707654145,north,Democrat,476256944,476256944,north,Democrat
10260,1351236267707654145,1351236267707654145,north,Democrat,13218102,13218102,south,Republican
10282,1351236267707654145,1351236267707654145,north,Democrat,1048784496,1048784496,north,Republican
10360,1356714265894408196,1356714265894408196,south,Democrat,1048784496,1048784496,north,Republican


In [23]:
# show duplicated rows
filtered_data[filtered_data.duplicated()]

,author_id,twitter_id,region,party,mentioned_id,twitter_id_mentioned,region_mentioned,party_mentioned
72,102477372,278124059,north,Democrat,131546062,131546062,south,Republican
116,3006258669,5558312,south,Republican,5558312,5558312,south,Republican
120,3006258669,5558312,south,Republican,5558312,5558312,south,Republican
125,3006258669,5558312,south,Republican,5558312,5558312,south,Republican
126,3006258669,5558312,south,Republican,5558312,5558312,south,Republican
...,...,...,...,...,...,...,...,...
10191,1346873610875654147,1346873610875654147,south,Republican,968650362,968650362,south,Republican
10194,1346873610875654147,1346873610875654147,south,Republican,109071031,109071031,south,Democrat
10199,1346873610875654147,1346873610875654147,south,Republican,968650362,968650362,south,Republican
10212,1351236267707654145,1351236267707654145,north,Democrat,476256944,476256944,north,Democrat


In [24]:
filtered_data = filtered_data.drop_duplicates()
filtered_data

,author_id,twitter_id,region,party,mentioned_id,twitter_id_mentioned,region_mentioned,party_mentioned
38,45645232,224285242,north,Democrat,82489457,1351236267707654145,north,Democrat
67,102477372,278124059,north,Democrat,131546062,131546062,south,Republican
112,3006258669,5558312,south,Republican,5558312,5558312,south,Republican
113,3006258669,5558312,south,Republican,382791093,382791093,north,Republican
171,3006258669,5558312,south,Republican,76456274,76456274,north,Democrat
...,...,...,...,...,...,...,...,...
10206,1351236267707654145,1351236267707654145,north,Democrat,1221242033530195970,1352287997853622273,south,Democrat
10208,1351236267707654145,1351236267707654145,north,Democrat,476256944,476256944,north,Democrat
10260,1351236267707654145,1351236267707654145,north,Democrat,13218102,13218102,south,Republican
10360,1356714265894408196,1356714265894408196,south,Democrat,1048784496,1048784496,north,Republican


In [25]:
len(filtered_data['author_id'].unique())

106

In [26]:
len(filtered_data['mentioned_id'].unique())

108

In [27]:
filtered_data.head(1)

,author_id,twitter_id,region,party,mentioned_id,twitter_id_mentioned,region_mentioned,party_mentioned
38,45645232,224285242,north,Democrat,82489457,1351236267707654145,north,Democrat


In [ ]:
import pandas as pd

# Assuming df is your original DataFrame
df = pd.DataFrame({
    'author_id': [45645232, 82489457, 3006258669, 3006258669, 3006258669],
    'mentioned_id': [82489457, 45645232, 5558312, 382791093, 76456274]
})

# Create a new DataFrame with the desired structure
new_df = pd.DataFrame({
    'user1': df['author_id'],
    'user2': df['mentioned_id'],
})

# Function to check if users mentioned each other
def check_mention(row):
    return any((df['author_id'] == row['user2']) & (df['mentioned_id'] == row['user1']))

# Apply the function to each row
new_df['mentions_each_other'] = new_df.apply(check_mention, axis=1)

# new_df['set'] = new_df.apply(lambda x: {x['user1'], x['user2']}, axis=1)

# new_df = new_df.drop_duplicates(subset=['set'])
new_df


In [ ]:
import pandas as pd

# Assuming df is your original DataFrame
df = filtered_data

# Create a new DataFrame with the desired structure
new_df = pd.DataFrame({
    'author_id': df['author_id'],
    'twitter_id': df['twitter_id'],
    'region': df['region'],
    'party': df['party'],
    'mentioned_id': df['mentioned_id'],
    'twitter_id_mentioned': df['twitter_id_mentioned'],
    'region_mentioned': df['region_mentioned'],
    'party_mentioned': df['party_mentioned']
})

# Function to check if users mentioned each other
def check_mention(row):
    return any(
        (df['author_id'] == row['mentioned_id']) & (df['mentioned_id'] == row['author_id'])
        )
    return any((df['twitter_id'] == row['twitter_id_mentioned']) & (df['twitter_id_mentioned'] == row['twitter_id']))

# Apply the function to each row
new_df['mentions_each_other'] = new_df.apply(check_mention, axis=1)

new_df

In [ ]:
# create set from author_id and mentioned_id columns into new column   
new_df['set'] = new_df.apply(lambda x: {x['twitter_id'], x['twitter_id_mentioned']}, axis=1)
# drop duplicates in set column
new_df = new_df.drop_duplicates(subset=['set'])
new_df

In [ ]:
# get unique values from party column
new_df['party'].unique()
# create different dataframes for each party being democrat, republican or independent
democrat = new_df[new_df['party'] == 'Democrat']
republican = new_df[new_df['party'] == 'Republican']
independent = new_df[new_df['party'] == 'Independent']
# create different dataframes for each region being north and south
north = new_df[new_df['region'] == 'north']
south = new_df[new_df['region'] == 'south']


In [ ]:
import pandas as pd
import numpy as np

# Assuming your DataFrame is named df
df = south

# Replace the boolean values with integers
df['mentions_each_other'] = df['mentions_each_other'].astype(int)

# Create pivot tables for each direction of interaction
pivot_table1 = df.pivot(index='twitter_id', columns='twitter_id_mentioned', values='mentions_each_other').fillna(0)
pivot_table2 = df.pivot(index='twitter_id_mentioned', columns='twitter_id', values='mentions_each_other').fillna(0)

# Combine pivot tables
interaction_matrix = pivot_table1.add(pivot_table2, fill_value=0)

# Ensure the matrix is symmetric (i.e., user1->user2 = user2->user1)
interaction_matrix = interaction_matrix.clip(upper=1)

# Fill NaN values with 0
interaction_matrix = interaction_matrix.fillna(0)

# Show the interaction matrix
interaction_matrix

In [ ]:
#create sql file
import sqlite3

conn = sqlite3.connect('csvyerine.db')
c = conn.cursor()

In [ ]:
# drop interaction_matrix table if exists
c.execute('''DROP TABLE IF EXISTS interaction_matrix''')

In [ ]:
#upload dataframe to sql
interaction_matrix.to_sql('mentionMatrixSouth', conn, if_exists='replace', index=True)

In [ ]:
# close connection
conn.close()

## retweet & quote

In [1]:
import sqlite3
conn = sqlite3.connect("D://Dropbox//projeler//ndsu_polarization//twitterVeriToplama//twitter004.db")
c = conn.cursor()
c

### create retweet and quoted accounts table (once)

In [ ]:
# create new table
c.execute('''
    CREATE TABLE retweetedAndQuotedUsers (
        author_id TEXT,
        twitter_id TEXT,
        retweetedOrQuoted_id TEXT,
        state TEXT,
        party TEXT
    )
''')

In [3]:
# query data from finalTweetsTable
c.execute('''SELECT author_id, twitter_id, state, party, in_reply_to_user_id,referenced_tweets 
FROM finalTweetsTable
WHERE referenced_tweets LIKE '%type=quoted%'
OR referenced_tweets LIKE '%type=retweeted%'
''')
rows = c.fetchall()

In [4]:
len(rows)

4782

In [5]:
rows[0]

('45645232',
 '224285242',
 'CO',
 'Democrat',
 None,
 '[<ReferencedTweet id=1587551549114519552 type=quoted>]')

In [6]:
import pandas as pd
import re

# Convert the rows to a DataFrame
columns = ['author_id', 'twitter_id', 'state', 'party', 'in_reply', 'referenced_tweets']
df = pd.DataFrame(rows, columns=columns)

# Extract ids from the referenced_tweets column and create a new column
def extract_id(text):
    match = re.search(r'id=(\d+)', text)
    if match:
        return match.group(1)
    return None

df['referenced_id'] = df['referenced_tweets'].apply(extract_id)

df

,author_id,twitter_id,state,party,in_reply,referenced_tweets,referenced_id
0,45645232,224285242,CO,Democrat,None,[<ReferencedTweet id=1587551549114519552 type=...,1587551549114519552
1,45645232,224285242,CO,Democrat,None,[<ReferencedTweet id=1585452562643267584 type=...,1585452562643267584
2,45645232,224285242,CO,Democrat,None,[<ReferencedTweet id=1583917447501197312 type=...,1583917447501197312
3,45645232,224285242,CO,Democrat,None,[<ReferencedTweet id=1578022273822375936 type=...,1578022273822375936
4,102477372,278124059,CT,Democrat,None,[<ReferencedTweet id=1590181901985517568 type=...,1590181901985517568
...,...,...,...,...,...,...,...
4777,1352287997853622273,1352287997853622273,GA,Democrat,None,[<ReferencedTweet id=1587441615626436609 type=...,1587441615626436609
4778,1352287997853622273,1352287997853622273,GA,Democrat,None,[<ReferencedTweet id=1585982855854931969 type=...,1585982855854931969
4779,1352287997853622273,1352287997853622273,GA,Democrat,None,[<ReferencedTweet id=1586101782874300417 type=...,1586101782874300417
4780,1352287997853622273,1352287997853622273,GA,Democrat,None,[<ReferencedTweet id=1584896350566715393 type=...,1584896350566715393


In [7]:
# df to csv
df.to_csv('retweetAndQuoteToGet.csv', index=False)

In [9]:
import requests
import pandas as pd
import logging
from tqdm import tqdm

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Your bearer token from the Twitter Developer Dashboard
BEARER_TOKEN = 'AAAAAAAAAAAAAAAAAAAAAK1XkAEAAAAAKpjRg7ZYOoJMdJ1pq5yoex0TPMA%3DAfFQNi09bSGjZPaiNP5XDHBeeS7US9pFvSSWnVzxyhqr3iAThw'

headers = {
    "Authorization": f"Bearer {BEARER_TOKEN}",
    "User-Agent": "v2RecentTweetLookupPython"
}

# Define a function to fetch user ID by Tweet ID
def get_user_id_by_tweet(tweet_id):
    endpoint = f"https://api.twitter.com/2/tweets/{tweet_id}"
    try:
        response = requests.get(endpoint, headers=headers).json()
        
        # Extract user ID from response
        user_id = response.get("data", {}).get("author_id", None)
        if user_id:
            logging.info(f"Successfully fetched user ID {user_id} for Tweet ID {tweet_id}")
        else:
            logging.warning(f"No user ID found for Tweet ID {tweet_id}. Response: {response}")
        return user_id
    except Exception as e:
        logging.error(f"Error fetching user ID for Tweet ID {tweet_id}. Exception: {e}")
        return None

# Sample DataFrame
df = df.head()

logging.info("Starting to fetch user IDs for each Tweet ID...")

# Fetch user IDs for each Tweet ID using tqdm for progress indication
df['referenced_user_id'] = [get_user_id_by_tweet(tweet_id) for tweet_id in tqdm(df['referenced_id'], desc="Fetching User IDs", unit="tweet")]

logging.info("Fetching process completed.")

df


2023-08-11 22:19:16,933 - INFO - Starting to fetch user IDs for each Tweet ID...
Fetching User IDs: 100%|██████████| 5/5 [00:01<00:00,  4.84tweet/s]
2023-08-11 22:19:17,969 - INFO - Fetching process completed.


,author_id,twitter_id,state,party,in_reply,referenced_tweets,referenced_id,referenced_user_id
0,45645232,224285242,CO,Democrat,None,[<ReferencedTweet id=1587551549114519552 type=...,1587551549114519552,None
1,45645232,224285242,CO,Democrat,None,[<ReferencedTweet id=1585452562643267584 type=...,1585452562643267584,None
2,45645232,224285242,CO,Democrat,None,[<ReferencedTweet id=1583917447501197312 type=...,1583917447501197312,None
3,45645232,224285242,CO,Democrat,None,[<ReferencedTweet id=1578022273822375936 type=...,1578022273822375936,None
4,102477372,278124059,CT,Democrat,None,[<ReferencedTweet id=1590181901985517568 type=...,1590181901985517568,None


In [ ]:
# get first 5 rows of the data


In [82]:
# Convert both columns to strings
df['referenced_id'] = df['referenced_id'].astype(str)
df['author_id'] = df['author_id'].astype(str)

# Filter rows where referenced_id is in author_id
df_filtered = df[df['referenced_id'].isin(df['author_id'])]

df_filtered


,author_id,twitter_id,state,party,referenced_tweets,referenced_id


In [83]:
print("Unique author_ids:", len(df['author_id'].unique()))
print("Unique referenced_ids:", len(df['referenced_id'].unique()))


Unique author_ids: 118
Unique referenced_ids: 4389


In [84]:
common_ids = set(df['referenced_id']).intersection(set(df['author_id']))
print("Common IDs:", len(common_ids))


Common IDs: 0


In [86]:
df_filtered = df[df['referenced_id'].isin(common_ids)]
print(df_filtered)


Empty DataFrame
Columns: [author_id, twitter_id, state, party, referenced_tweets, referenced_id]
Index: []


In [ ]:
# parse JSON and insert data into new table
import json
for row in rows:
    author_id, twitter_id, state, party, entities = row
    if entities is not None:
        try:
            entities_json = json.loads(entities)
            for mention in entities_json.get('mentions', []):
                mentioned_id = mention.get('id')
                c.execute('INSERT INTO MentionedUsers3 (author_id, twitter_id, state, party, mentioned_id) VALUES (?, ?, ?, ?, ?)', (author_id, twitter_id, state, party, mentioned_id))
        except json.JSONDecodeError:
            print(f"Error parsing JSON for author_id {author_id}: {entities}")

# commit the transactions
conn.commit()

### create matrix

In [ ]:
# read * from mentionedUsers table to df
import pandas as pd
mentionedUsers = pd.read_sql_query("SELECT * FROM mentionedUsers3", conn)
#drop state column
mentionedUsers = mentionedUsers.drop(columns=['state'])
mentionedUsers
# burada id sayısı çok fazla çünkü tüm mentionlananlar var. biz sadece kendi listemizdekileri alacağız.

,author_id,twitter_id,mentioned_id,party,region
0,45645232,224285242,4750779374,Democrat,north
1,45645232,224285242,483010877,Democrat,north
2,45645232,224285242,117839957,Democrat,north
3,45645232,224285242,540352897,Democrat,north
4,45645232,224285242,228591530,Democrat,north
...,...,...,...,...,...
10392,1352287997853622273,1352287997853622273,123935925,Democrat,south
10393,1352287997853622273,1352287997853622273,14275261,Democrat,south
10394,1352287997853622273,1352287997853622273,957024441338417154,Democrat,south
10395,1352287997853622273,1352287997853622273,44783853,Democrat,south


In [ ]:
authorIDTwitterID = pd.read_sql_query("SELECT * FROM authorIDTwitterID", conn)
authorIDTwitterID = authorIDTwitterID[['author_id', 'twitter_id', 'region', 'party']]
authorIDTwitterID

,author_id,twitter_id,region,party
0,1017500185356853248,1017500185356853248,south,Republican
1,102477372,278124059,north,Democrat
2,1048784496,1048784496,north,Republican
3,1058520120,1058520120,north,Democrat
4,1060370282,1060370282,south,Republican
...,...,...,...,...
119,941000686275387392,941000686275387392,north,Democrat
120,946549322,946549322,north,Democrat
121,968650362,968650362,south,Republican
122,970207298,970207298,north,Democrat


In [ ]:
conn.close()

In [ ]:
# mentionedUsers.merge(authorIDTwitterID, left_on = 'mentioned_id', right_on = 'author_id', how='left')
mentionedUsers = mentionedUsers.merge(authorIDTwitterID, left_on='mentioned_id', right_on='author_id', how='left', suffixes=('', '_mentioned'))

In [ ]:
mentionedUsers

,author_id,twitter_id,mentioned_id,party,region,author_id_mentioned,twitter_id_mentioned,region_mentioned,party_mentioned
0,45645232,224285242,4750779374,Democrat,north,NaN,NaN,NaN,NaN
1,45645232,224285242,483010877,Democrat,north,NaN,NaN,NaN,NaN
2,45645232,224285242,117839957,Democrat,north,NaN,NaN,NaN,NaN
3,45645232,224285242,540352897,Democrat,north,NaN,NaN,NaN,NaN
4,45645232,224285242,228591530,Democrat,north,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
10392,1352287997853622273,1352287997853622273,123935925,Democrat,south,NaN,NaN,NaN,NaN
10393,1352287997853622273,1352287997853622273,14275261,Democrat,south,NaN,NaN,NaN,NaN
10394,1352287997853622273,1352287997853622273,957024441338417154,Democrat,south,NaN,NaN,NaN,NaN
10395,1352287997853622273,1352287997853622273,44783853,Democrat,south,NaN,NaN,NaN,NaN


In [ ]:
print(len(mentionedUsers["mentioned_id"].unique()))
print(len(mentionedUsers["author_id"].unique()))
print(len(mentionedUsers["twitter_id"].unique()))


3479
121
90


In [ ]:
# what are the number of unique elements in author_id different than twitter_id?
print(len(mentionedUsers["author_id"].unique()) - len(mentionedUsers["twitter_id"].unique()))

31


In [ ]:
# get unique values from 'author_id' column
unique_user1 = mentionedUsers['author_id'].unique()

# filter 'mentioned_id' column based on these unique values
filtered_data = mentionedUsers[mentionedUsers['mentioned_id'].isin(unique_user1)]

In [ ]:
print(len(filtered_data))
# buradaki sayı finalTable'daki sayıdan az ama normal. herkes birilerini mentionlamıyor.
print(len(filtered_data['author_id'].unique()))
print(len(filtered_data['twitter_id'].unique()))
print(len(filtered_data['mentioned_id'].unique()))


1805
108
88
113


In [ ]:
filtered_data

,author_id,twitter_id,mentioned_id,party,region,author_id_mentioned,twitter_id_mentioned,region_mentioned,party_mentioned
13,45645232,224285242,45645232,Democrat,north,45645232,224285242,north,Democrat
38,45645232,224285242,82489457,Democrat,north,82489457,1351236267707654145,north,Democrat
41,102477372,278124059,102477372,Democrat,north,102477372,278124059,north,Democrat
43,102477372,278124059,102477372,Democrat,north,102477372,278124059,north,Democrat
45,102477372,278124059,102477372,Democrat,north,102477372,278124059,north,Democrat
...,...,...,...,...,...,...,...,...,...
10314,1356714265894408196,1356714265894408196,1356714265894408196,Democrat,south,1356714265894408196,1356714265894408196,south,Democrat
10320,1356714265894408196,1356714265894408196,1356714265894408196,Democrat,south,1356714265894408196,1356714265894408196,south,Democrat
10354,1356714265894408196,1356714265894408196,1356714265894408196,Democrat,south,1356714265894408196,1356714265894408196,south,Democrat
10360,1356714265894408196,1356714265894408196,1048784496,Democrat,south,1048784496,1048784496,north,Republican


In [ ]:
(filtered_data["mentioned_id"] == filtered_data["author_id_mentioned"]).sum()

1805

In [ ]:
# drop author_id_mentioned column from filtered_data
filtered_data = filtered_data.drop(columns=['author_id_mentioned'])

In [ ]:
# rearrange columns
filtered_data = filtered_data[['author_id', 'twitter_id', 'region', 'party', 'mentioned_id', 'twitter_id_mentioned', 'region_mentioned', 'party_mentioned']]
filtered_data

,author_id,twitter_id,region,party,mentioned_id,twitter_id_mentioned,region_mentioned,party_mentioned
13,45645232,224285242,north,Democrat,45645232,224285242,north,Democrat
38,45645232,224285242,north,Democrat,82489457,1351236267707654145,north,Democrat
41,102477372,278124059,north,Democrat,102477372,278124059,north,Democrat
43,102477372,278124059,north,Democrat,102477372,278124059,north,Democrat
45,102477372,278124059,north,Democrat,102477372,278124059,north,Democrat
...,...,...,...,...,...,...,...,...
10314,1356714265894408196,1356714265894408196,south,Democrat,1356714265894408196,1356714265894408196,south,Democrat
10320,1356714265894408196,1356714265894408196,south,Democrat,1356714265894408196,1356714265894408196,south,Democrat
10354,1356714265894408196,1356714265894408196,south,Democrat,1356714265894408196,1356714265894408196,south,Democrat
10360,1356714265894408196,1356714265894408196,south,Democrat,1048784496,1048784496,north,Republican


In [ ]:
# kendini mentionlayanları çıkar
filtered_data = filtered_data[filtered_data['author_id'] != filtered_data['mentioned_id']]
filtered_data

,author_id,twitter_id,region,party,mentioned_id,twitter_id_mentioned,region_mentioned,party_mentioned
38,45645232,224285242,north,Democrat,82489457,1351236267707654145,north,Democrat
67,102477372,278124059,north,Democrat,131546062,131546062,south,Republican
72,102477372,278124059,north,Democrat,131546062,131546062,south,Republican
112,3006258669,5558312,south,Republican,5558312,5558312,south,Republican
113,3006258669,5558312,south,Republican,382791093,382791093,north,Republican
...,...,...,...,...,...,...,...,...
10212,1351236267707654145,1351236267707654145,north,Democrat,476256944,476256944,north,Democrat
10260,1351236267707654145,1351236267707654145,north,Democrat,13218102,13218102,south,Republican
10282,1351236267707654145,1351236267707654145,north,Democrat,1048784496,1048784496,north,Republican
10360,1356714265894408196,1356714265894408196,south,Democrat,1048784496,1048784496,north,Republican


In [ ]:
# show duplicated rows
filtered_data[filtered_data.duplicated()]

,author_id,twitter_id,region,party,mentioned_id,twitter_id_mentioned,region_mentioned,party_mentioned
72,102477372,278124059,north,Democrat,131546062,131546062,south,Republican
116,3006258669,5558312,south,Republican,5558312,5558312,south,Republican
120,3006258669,5558312,south,Republican,5558312,5558312,south,Republican
125,3006258669,5558312,south,Republican,5558312,5558312,south,Republican
126,3006258669,5558312,south,Republican,5558312,5558312,south,Republican
...,...,...,...,...,...,...,...,...
10191,1346873610875654147,1346873610875654147,south,Republican,968650362,968650362,south,Republican
10194,1346873610875654147,1346873610875654147,south,Republican,109071031,109071031,south,Democrat
10199,1346873610875654147,1346873610875654147,south,Republican,968650362,968650362,south,Republican
10212,1351236267707654145,1351236267707654145,north,Democrat,476256944,476256944,north,Democrat


In [ ]:
filtered_data = filtered_data.drop_duplicates()
filtered_data

,author_id,twitter_id,region,party,mentioned_id,twitter_id_mentioned,region_mentioned,party_mentioned
38,45645232,224285242,north,Democrat,82489457,1351236267707654145,north,Democrat
67,102477372,278124059,north,Democrat,131546062,131546062,south,Republican
112,3006258669,5558312,south,Republican,5558312,5558312,south,Republican
113,3006258669,5558312,south,Republican,382791093,382791093,north,Republican
171,3006258669,5558312,south,Republican,76456274,76456274,north,Democrat
...,...,...,...,...,...,...,...,...
10206,1351236267707654145,1351236267707654145,north,Democrat,1221242033530195970,1352287997853622273,south,Democrat
10208,1351236267707654145,1351236267707654145,north,Democrat,476256944,476256944,north,Democrat
10260,1351236267707654145,1351236267707654145,north,Democrat,13218102,13218102,south,Republican
10360,1356714265894408196,1356714265894408196,south,Democrat,1048784496,1048784496,north,Republican


In [ ]:
len(filtered_data['author_id'].unique())

106

In [ ]:
len(filtered_data['mentioned_id'].unique())

108

In [ ]:
filtered_data.head(1)

,author_id,twitter_id,region,party,mentioned_id,twitter_id_mentioned,region_mentioned,party_mentioned
38,45645232,224285242,north,Democrat,82489457,1351236267707654145,north,Democrat


In [ ]:
import pandas as pd

# Assuming df is your original DataFrame
df = pd.DataFrame({
    'author_id': [45645232, 82489457, 3006258669, 3006258669, 3006258669],
    'mentioned_id': [82489457, 45645232, 5558312, 382791093, 76456274]
})

# Create a new DataFrame with the desired structure
new_df = pd.DataFrame({
    'user1': df['author_id'],
    'user2': df['mentioned_id'],
})

# Function to check if users mentioned each other
def check_mention(row):
    return any((df['author_id'] == row['user2']) & (df['mentioned_id'] == row['user1']))

# Apply the function to each row
new_df['mentions_each_other'] = new_df.apply(check_mention, axis=1)

# new_df['set'] = new_df.apply(lambda x: {x['user1'], x['user2']}, axis=1)

# new_df = new_df.drop_duplicates(subset=['set'])
new_df


In [ ]:
import pandas as pd

# Assuming df is your original DataFrame
df = filtered_data

# Create a new DataFrame with the desired structure
new_df = pd.DataFrame({
    'author_id': df['author_id'],
    'twitter_id': df['twitter_id'],
    'region': df['region'],
    'party': df['party'],
    'mentioned_id': df['mentioned_id'],
    'twitter_id_mentioned': df['twitter_id_mentioned'],
    'region_mentioned': df['region_mentioned'],
    'party_mentioned': df['party_mentioned']
})

# Function to check if users mentioned each other
def check_mention(row):
    return any(
        (df['author_id'] == row['mentioned_id']) & (df['mentioned_id'] == row['author_id'])
        )
    return any((df['twitter_id'] == row['twitter_id_mentioned']) & (df['twitter_id_mentioned'] == row['twitter_id']))

# Apply the function to each row
new_df['mentions_each_other'] = new_df.apply(check_mention, axis=1)

new_df

In [ ]:
# create set from author_id and mentioned_id columns into new column   
new_df['set'] = new_df.apply(lambda x: {x['twitter_id'], x['twitter_id_mentioned']}, axis=1)
# drop duplicates in set column
new_df = new_df.drop_duplicates(subset=['set'])
new_df

In [ ]:
# get unique values from party column
new_df['party'].unique()
# create different dataframes for each party being democrat, republican or independent
democrat = new_df[new_df['party'] == 'Democrat']
republican = new_df[new_df['party'] == 'Republican']
independent = new_df[new_df['party'] == 'Independent']
# create different dataframes for each region being north and south
north = new_df[new_df['region'] == 'north']
south = new_df[new_df['region'] == 'south']


In [ ]:
import pandas as pd
import numpy as np

# Assuming your DataFrame is named df
df = south

# Replace the boolean values with integers
df['mentions_each_other'] = df['mentions_each_other'].astype(int)

# Create pivot tables for each direction of interaction
pivot_table1 = df.pivot(index='twitter_id', columns='twitter_id_mentioned', values='mentions_each_other').fillna(0)
pivot_table2 = df.pivot(index='twitter_id_mentioned', columns='twitter_id', values='mentions_each_other').fillna(0)

# Combine pivot tables
interaction_matrix = pivot_table1.add(pivot_table2, fill_value=0)

# Ensure the matrix is symmetric (i.e., user1->user2 = user2->user1)
interaction_matrix = interaction_matrix.clip(upper=1)

# Fill NaN values with 0
interaction_matrix = interaction_matrix.fillna(0)

# Show the interaction matrix
interaction_matrix

In [ ]:
#create sql file
import sqlite3

conn = sqlite3.connect('csvyerine.db')
c = conn.cursor()

In [ ]:
# drop interaction_matrix table if exists
c.execute('''DROP TABLE IF EXISTS interaction_matrix''')

In [ ]:
#upload dataframe to sql
interaction_matrix.to_sql('mentionMatrixSouth', conn, if_exists='replace', index=True)

In [ ]:
# close connection
conn.close()